In [7]:
'''
Script that runs autocombinatorial generation of new molecules from an initial non-substituted
bipyriding.

1) is cuts outs hydrogen at specffied positions of bpy and then pastes new functional groups instead

2) it geometry optimizes new molecule using MMFF94 force field

The process for optimization requires a MOL file so conversion of xyz to mol was done using obabel

Author: Claire Anderson
'''

import fileinput
import random
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdmolfiles

# starting input file, vanilla bpy
bpy = 'bpy.mol'

# input how mant autocombinatorial outputs you want from this file
print('how many autocombinatorial outputs do you want?')
number_of_outputs = input()
number_of_outputs = int(number_of_outputs)


hydrogen1 = ''
hydrogen2 = ''
chlorine1 = '    2.0625   -1.4289    0.0000 Cl  0  0  0  0  0  0  0  0  0  0  0  0'
chlorine2 = '   -2.0625   -1.4289    0.0000 Cl  0  0  0  0  0  0  0  0  0  0  0  0'
methane1  = '    2.0625   -1.4289    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0'
methane2  = '   -2.0625   -1.4289    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0'
alcohol1  = '    2.1479   -1.4289    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0'
alcohol2  = '   -2.1479   -1.4289    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0'
CHO1      = '    2.0625   -2.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0' + '\n' '    2.6500   -2.7862    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0'
CHO2      = '   -2.0625   -2.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0' + '\n' '   -2.6500   -2.7862    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0'
OCH31     = '    2.0625   -2.0717    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0' + '\n' '    2.8875   -2.7862    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0'                                                    
OCH32     = '   -2.0625   -2.0717    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0' + '\n' '   -2.8875   -2.7862    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0'                                                                                          
NCH31     = '    2.0625   -2.0717    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0' + '\n' '    2.8875   -2.7862    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0'                                                                        
NCH32     = '   -2.0625   -2.0717    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0' + '\n' '   -2.8875   -2.7862    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0'                                                                        
COOH1     = '    2.0625   -2.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0' + '\n' '    2.8875   -2.7862    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0' + '\n' '    1.6500   -2.7862    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0'                                                                                    
COOH2     = '   -2.0625   -2.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0' + '\n' '   -2.8875   -2.7862    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0' + '\n' '   -1.6500   -2.7862    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0'                                                                                    
nitro1    = '    2.0625   -2.0305    0.0000 N   0  3  0  0  0  0  0  0  0  0  0  0' + '\n' '    2.8875   -2.0305    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0' + '\n' '    1.8490   -2.8305    0.0000 O   0  5  0  0  0  0  0  0  0  0  0  0'
nitro2    = '   -2.0625   -2.0305    0.0000 N   0  3  0  0  0  0  0  0  0  0  0  0' + '\n' '   -2.8875   -2.0305    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0' + '\n' '   -1.8490   -2.8305    0.0000 O   0  5  0  0  0  0  0  0  0  0  0  0'


string1 = (
            hydrogen1,
            chlorine1,
            methane1,
            alcohol1,
            CHO1,
            OCH31,
            NCH31,
            COOH1,
            nitro1,
            )
string2 = (
            hydrogen2,
            chlorine2,
            methane2,
            alcohol2,
            CHO2,
            OCH32,
            NCH32,
            COOH2,
            nitro2,
            )

for i in range(1, number_of_outputs+1):
    
    # define all the file names necessary in conversions 
    output_filename = str(i) + '.mol'
    opt_filename = str(i) + 'opt.mol'
    optimized_filename = str(i) + 'optimized.mol'
    converted_filename = str(i) + 'converted.inp' 
    trim_filename = str(i) + 'trim.inp'
    final_filename = str(i) + 'final.inp'
    
    # create all files necessary for autocombinatorial output and conversions
    output_file = open(output_filename, mode="w",encoding="utf-8")
    trim_file = open(trim_filename, mode="w",encoding="utf-8")
    final_file = open(final_filename, mode="w",encoding="utf-8")
    
    # define functional group replacements

    for line in fileinput.FileInput(bpy,inplace=0):
        output_file.write(line)
        if ' 0.4125    1.0717    0.0000 C' in line:
            
            choice1 = random.choice(string1)
            if choice1 != hydrogen1:
                output_file.write(choice1 +'\n')

            choice2 = random.choice(string2)
            if choice2 != hydrogen2:
                output_file.write(choice2 + '\n')
                
                
                
        if ' 13 14  1  0' in line:
            if choice1 != hydrogen1:
                output_file.write(' 10 15  1  0' + '\n')
            if choice1 == CHO1:
                output_file.write(' 15 16  2  0' + '\n')
            if choice1 == OCH31 or choice1 == NCH31:
                output_file.write(' 15 16  1  0' + '\n')
            if choice1 == COOH1:
                output_file.write(' 15 16  1  0' + '\n')
                output_file.write(' 15 17  2  0' + '\n')
            if choice1 == nitro1:
                output_file.write(' 15 16  2  0' + '\n')
                output_file.write(' 15 17  1  0' + '\n')
            
            if choice2 != hydrogen2:
                if choice1 == hydrogen1:
                    output_file.write('  2 15  1  0' +'\n')
                    if choice2 == CHO2:
                        output_file.write(' 15 16  2  0' + '\n')
                    if choice2 == OCH32 or choice2 == NCH32:
                        output_file.write(' 15 16  1  0' + '\n')
                    if choice2 == COOH2:
                        output_file.write(' 15 16  1  0' + '\n')
                        output_file.write(' 15 17  2  0' + '\n')
                    if choice2 == nitro2:
                        output_file.write(' 15 16  2  0' + '\n')
                        output_file.write(' 15 17  1  0' + '\n')
                        output_file.write('M  CHG  2  15   1  17  -1' +'\n')
                elif choice1 == CHO1 or choice1 == OCH31 or choice1 == NCH31:
                    output_file.write('  2 17  1  0' + '\n')
                    if choice2 == CHO2:
                        output_file.write(' 17 18  2  0' + '\n')
                    if choice2 == OCH32 or choice2 == NCH32:
                        output_file.write(' 17 18  1  0' + '\n')
                    if choice2 == COOH2:
                        output_file.write(' 17 18  1  0' + '\n')
                        output_file.write(' 17 19  2  0' + '\n')
                    if choice2 == nitro2:
                        output_file.write(' 17 18  2  0' + '\n')
                        output_file.write(' 17 19  1  0' + '\n')
                        output_file.write('M  CHG  2  17   1  19  -1' +'\n')
                elif choice1 == COOH1 or choice1 == nitro1:
                    output_file.write('  2 18  1  0' + '\n')
                    if choice2 == CHO2:
                        output_file.write(' 18 19  2  0' + '\n')
                    if choice2 == OCH32 or choice2 == NCH32:
                        output_file.write(' 18 19  1  0' + '\n')
                    if choice2 == COOH2:
                        output_file.write(' 18 19  1  0' + '\n')
                        output_file.write(' 18 20  2  0' + '\n')
                    if choice2 == nitro2 and choice1 != nitro1:
                        output_file.write(' 18 19  2  0' + '\n')
                        output_file.write(' 18 20  1  0' + '\n')
                        output_file.write('M  CHG  2  18   1  20  -1' +'\n')
                    if choice2 == nitro2 and choice1 == nitro1:
                        output_file.write(' 18 19  2  0' + '\n')
                        output_file.write(' 18 20  1  0' + '\n')
                        output_file.write('M  CHG  4  15   1  17  -1  18   1  20  -1' +'\n')
                else:
                    if choice2 == alcohol2 or choice2 == methane2 or choice2 == chlorine2 or choice2 == amine2:
                        output_file.write('  2 16  1  0' +'\n')
                    if choice2 == CHO2: 
                        output_file.write('  2 16  1  0' + '\n')
                        output_file.write(' 16 17  2  0' + '\n')
                    if choice2 == OCH32 or choice2 == NCH32:
                        output_file.write('  2 16  1  0' + '\n')
                        output_file.write(' 16 17  1  0' + '\n')
                    if choice2 == COOH2:
                        output_file.write('  2 16  1  0' + '\n')
                        output_file.write(' 16 17  1  0' + '\n')
                        output_file.write(' 16 18  2  0' + '\n')
                    if choice2 == nitro2:
                        output_file.write('  2 16  1  0' + '\n')
                        output_file.write(' 16 17  2  0' + '\n')
                        output_file.write(' 16 18  1  0' + '\n')
                        output_file.write('M  CHG  2  16   1  18  -1' +'\n')
    output_file.close()
    
    #recopy output file but with correct numbering of atoms and bonds
    output_file = open(output_filename, mode="r",encoding="utf-8")
    count_atoms = 0
    number_bonds = 0
    for line in fileinput.FileInput(output_filename, inplace=0):
        if ' C   ' in line or  ' N   ' in line or  ' Cl  ' in line or  ' O   ' in line:
            count_atoms=count_atoms+1
        number_bonds=count_atoms+1
    output_file.close()
    
    opt_file = open(opt_filename, mode='w',encoding='utf-8')
    for line in fileinput.FileInput(output_filename,inplace=0):
        if ' 0999' in line:
            line = line.replace(line, ' ' + str(count_atoms) + ' ' + str(number_bonds) + '  0  0  0  0  0  0  0  0999 V2000' + '\n')
        opt_file.write(line)
    opt_file.close()
    
    m = Chem.MolFromMolFile(opt_filename)
    m = Chem.AddHs(m)
    molecule=Chem.MolToMolBlock(m)
    print(molecule)
    AllChem.MMFFOptimizeMolecule(m)
    Chem.rdmolfiles.MolToMolFile(m,optimized_filename)

    
    os.system('obabel ' + optimized_filename + ' -O ' + converted_filename)

    for line in fileinput.FileInput(converted_filename, inplace=0):
        if '$CONTRL' in line:
            line = line.replace(line, ' $CONTRL SCFTYP=RHF RUNTYP=HESSIAN EXETYP=RUN' + '\n' ' MAXIT=200 ICHARG=0 MULT=1 ISPHER=1' +'\n' ' COORD=CART UNITS=ANGS $END')
            final_file.write(line)
    final_file.write(line)
    final_file.write(' $BASIS GBASIS=CCD $END' +'\n')
    final_file.write(' $SYSTEM TIMLIM=1300 MWORDS=100 PARALL=.TRUE. $END'+'\n')
    final_file.write(' $SCF DIRSCF=.TRUE. $END '+'\n')
    final_file.write(' $STATPT HESS=READ $END'+'\n')
    for line in fileinput.FileInput(converted_filename, inplace=0):
        if ' $CONTRL COORD=CART' not in line:
            final_file.write(line)
    final_file.close()

#os.system('rm *mol')
#os.system('rm *converted*')
#os.system('rm *trim*')

how many autocombinatorial outputs do you want?
300
bpy.mol
     RDKit          2D

 32 33  0  0  0  0  0  0  0  0999 V2000
   -2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125    1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250    0.3572    0.0000 N   0

bpy.mol
     RDKit          2D

 29 30  0  0  0  0  0  0  0  0999 V2000
   -2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125    1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125    1.07

bpy.mol
     RDKit          2D

 34 35  0  0  0  0  0  0  0  0999 V2000
   -2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125    1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125    1.07

bpy.mol
     RDKit          2D

 30 31  0  0  0  0  0  0  0  0999 V2000
   -2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125    1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125    1.07

bpy.mol
     RDKit          2D

 36 37  0  0  0  0  0  0  0  0999 V2000
   -2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125    1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125    1.07

...

bpy.mol
     RDKit          2D

 30 31  0  0  0  0  0  0  0  0999 V2000
   -2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125    1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125    1.07

bpy.mol
     RDKit          2D

 30 31  0  0  0  0  0  0  0  0999 V2000
   -2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125    1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125    1.07

bpy.mol
     RDKit          2D

 31 32  0  0  0  0  0  0  0  0999 V2000
   -2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125    1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125    1.07

bpy.mol
     RDKit          2D

 32 33  0  0  0  0  0  0  0  0999 V2000
   -2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125    1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125    1.07

bpy.mol
     RDKit          2D

 35 36  0  0  0  0  0  0  0  0999 V2000
   -2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4125    1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500   -1.0717    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.0625   -0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.6500    0.3572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.8250    0.3572    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
    0.4125    1.07

# 